In [1]:
import numpy as np
import torch
from tqdm import tqdm
from recbole.quick_start.quick_start import load_data_and_model
from recbole.utils.case_study import full_sort_topk, full_sort_scores
import pandas as pd
import os
model_path='ADMMSLIM-Feb-20-2024_06-42-05.pth'
topk=10
save_path='submission.csv'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file=os.path.join('saved',model_path))

df = pd.read_csv('/home/minseo/Project/Recbole/data/train/train_ratings.csv')
user_grp = dict(df.groupby('user').item.apply(list))

submission = pd.read_csv('/home/minseo/Project/Recbole/data/eval/sample_submission.csv')
sub_user_idx = submission['user'].unique()
sub_user_idx = np.array(sub_user_idx,dtype=str)
uid_series = dataset.token2id(dataset.uid_field, sub_user_idx)
total_topk_score, total_topk_iid_list = torch.zeros_like(torch.Tensor(31360, topk)), torch.zeros_like(torch.Tensor(31360, topk))

for idx in tqdm(range(0,len(uid_series))):
    need_inf = dataset.token2id(dataset.iid_field, np.array(user_grp[int(sub_user_idx[idx])],dtype=str))
    mask = [True if i in need_inf else False for i in range(0,6808)]
    scores = full_sort_scores(np.array([uid_series[idx]]),model,valid_data,config['device'])
    new_scores=scores.cpu().masked_fill(torch.from_numpy(np.array(mask)),float('-inf'))    
    total_topk_score[idx] = torch.topk(new_scores,topk)[0]
    total_topk_iid_list[idx] = torch.topk(new_scores,topk)[1]
    
int_iid = total_topk_iid_list.to(torch.int64)
external_item_list = dataset.id2token(dataset.iid_field, int_iid.cpu())
external_item_list = external_item_list.flatten()
df = pd.DataFrame({'user': np.repeat(sub_user_idx, topk), 'item': external_item_list})
df.to_csv(save_path,index=False)


/home/minseo/anaconda3/envs/Recbole/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-20 09:52:47,217	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
saved_val = valid_data

torch.Size([302]) torch.Size([302])
torch.Size([144]) torch.Size([144])


20 Feb 09:26    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ML
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = Recall@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:

(The batch_size of interaction: 4096
    user_id, torch.Size([4096]), cpu, torch.int64
    item_id, torch.Size([4096]), cpu, torch.int64
    timestamp, torch.Size([4096]), cpu, torch.float32
    item_length, torch.Size([4096]), cpu, torch.int64
    item_id_list, torch.Size([4096, 50]), cpu, torch.int64
    timestamp_list, torch.Size([4096, 50]), cpu, torch.float32

, None, tensor([   0,    1,    2,  ..., 4093, 4094, 4095]), tensor([ 985, 4683,  805,  ..., 1678, 1348, 1321]))


In [ ]:
df = pd.read_csv('/home/minseo/Project/Recbole/data/train/train_ratings.csv')
user_grp = dict(df.groupby('user').item.apply(list))

submission = pd.read_csv('/home/minseo/Project/Recbole/data/eval/sample_submission.csv')
sub_user_idx = submission['user'].unique()
sub_user_idx = np.array(sub_user_idx,dtype=str)
uid_series = dataset.token2id(dataset.uid_field, sub_user_idx)
total_topk_score, total_topk_iid_list = torch.zeros_like(torch.Tensor(31360, topk)), torch.zeros_like(torch.Tensor(31360, topk))

for idx in tqdm(range(0,len(uid_series))):
    need_inf = dataset.token2id(dataset.iid_field, np.array(user_grp[int(sub_user_idx[idx])],dtype=str))
    mask = [True if i in need_inf else False for i in range(0,6808)]
    scores = full_sort_scores(np.array([uid_series[idx]]),model,saved_val,config['device'])
    new_scores=scores.cpu().masked_fill(torch.from_numpy(np.array(mask)),float('-inf'))    
    total_topk_score[idx] = torch.topk(new_scores,topk)[0]
    total_topk_iid_list[idx] = torch.topk(new_scores,topk)[1]
    
int_iid = total_topk_iid_list.to(torch.int64)
external_item_list = dataset.id2token(dataset.iid_field, int_iid.cpu())
external_item_list = external_item_list.flatten()
df = pd.DataFrame({'user': np.repeat(sub_user_idx, topk), 'item': external_item_list})
df.to_csv(save_path,index=False)

  0%|          | 0/31360 [00:00<?, ?it/s]


KeyError: 'item_id_list'

In [ ]:
new_scores.shape

torch.Size([37, 6808])